In [9]:
import os
import radiomics
from radiomics import featureextractor
import numpy as np
import nrrd

# Percorsi delle cartelle
pazienti_dir = 'C:\\Users\\bsbar\\Desktop\\pazienti_nrrd'
roi_dir = 'C:\\Users\\bsbar\\Desktop\\Tesi\\ROI'

cartelle_da_saltare = ['19', '40', '88', '150', '193', '107']

# Lista per i vettori delle features e i nomi dei pazienti
feature_vectors = []
patient_names = []
dimension_mismatch = []

# Imposta l'estrattore con le configurazioni di default
estrattore = featureextractor.RadiomicsFeatureExtractor()

# Funzione per trovare il file .nrrd in una cartella
def trova_nrrd_file(cartella):
    for file in os.listdir(cartella):
        if file.endswith('.nrrd'):
            return os.path.join(cartella, file)
    return None

# Funzione per verificare le dimensioni delle immagini
def verifica_dimensioni(immagine_file, maschera_file):
    immagine, _ = nrrd.read(immagine_file)
    maschera, _ = nrrd.read(maschera_file)
    return immagine.shape == maschera.shape

# Itera su tutti i pazienti
for paziente in sorted(os.listdir(pazienti_dir)):
    print(paziente)
    if paziente in cartelle_da_saltare:
        print(f"Saltata la cartella del paziente {paziente}")
        continue  # Salta questa iterazione e passa al paziente successivo
    
    # Verifica se la cartella esiste anche per le ROI
    roi_paziente_dir = os.path.join(roi_dir, paziente)
    paziente_dir = os.path.join(pazienti_dir, paziente)

    if os.path.isdir(roi_paziente_dir):
        # Trova i file immagine e maschera nella rispettiva cartella
        immagine_file = trova_nrrd_file(paziente_dir)
        maschera_file = trova_nrrd_file(roi_paziente_dir)
     
        if immagine_file and maschera_file:
            # Verifica se le dimensioni delle immagini e delle ROI corrispondono
            if verifica_dimensioni(immagine_file, maschera_file):
                # Aggiungi il nome del paziente alla lista
                patient_names.append(paziente)

                # Estrai le features radiomiche
                results = estrattore.execute(immagine_file, maschera_file)

                # Trasforma i risultati in un vettore di features
                feature_vector = np.array([value.item() if isinstance(value, np.ndarray) else value 
                           for key, value in results.items() 
                           if isinstance(value, (float, int, np.ndarray))])
                
                # Aggiungi il vettore delle features alla lista
                feature_vectors.append(feature_vector)
                
            else:
                # Aggiungi il nome del paziente e un messaggio di errore alla lista delle dimensioni non corrispondenti
                dimension_mismatch.append(paziente)

# Trasforma la lista di vettori in un array numpy
feature_matrix = np.array(feature_vectors)

# Ora hai:
# - 'feature_matrix': una matrice dove ogni riga è il vettore di features di un paziente
# - 'patient_names': una lista dei nomi dei pazienti corrispondente
# - 'dimension_mismatch': una lista dei nomi dei pazienti con dimensioni non corrispondenti

100


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


101


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


102


...

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


96


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


98


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


In [26]:
import csv

# Ordina i pazienti in base al loro nome numerico
patient_names_sorted, feature_vectors_sorted = zip(*sorted(zip(patient_names, feature_vectors), key=lambda x: int(x[0])))
feature_names = [key for key, value in results.items() if isinstance(value, (float, int, np.ndarray))]


# Salva le features e i nomi dei pazienti in un file CSV
output_csv = 'features_radiomiche.csv'
with open(output_csv, mode='w', newline='') as file:
    writer = csv.writer(file, delimiter=';')  # Usa il delimitatore punto e virgola

    # Scrivi l'intestazione: la prima colonna è "Paziente" e le altre colonne rappresentano le features
    writer.writerow(['Paziente'] + feature_names)

    # Scrivi i dati: il nome del paziente e le sue features
    for name, features in zip(patient_names_sorted, feature_vectors_sorted):
        writer.writerow([name] + list(features))

print(f'File CSV salvato come {output_csv}')

File CSV salvato come features_radiomiche.csv


In [36]:
import radiomics
from radiomics import featureextractor, getFeatureClasses

# Ottieni le classi di feature disponibili
feature_classes = getFeatureClasses()
print("Feature classes disponibili:")
for feature_class_name in feature_classes:
    print(feature_class_name)

Feature classes disponibili:
firstorder
glcm
gldm
glrlm
glszm
ngtdm
shape
shape2D
